# Spatial$^{*}$ Microsimulation Urban Metabolism Model (SMUM)

<div class="image123">
    <div class="imgContainer">
        <img src="./logos/UNEnvironment.png" alt="UNEP logo" style="width:200px">
    </div>
    <div class="imgContainer">
        <img src="./logos/GI-REC.png" alt="GI_REC logo" style="width:200px">
    </div>
</div>

# 1.c Micro-level Electricity demand model

[UN Environment](http://www.unep.org/)

In [1]:
import datetime; print(datetime.datetime.now())

2017-10-25 14:39:00.570024


**Notebook Abstract:**

A simple micro-level electricity deman model.

## Prior electricity demand model

In [16]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from urbanmetabolism._scripts.micro import compute_categories, change_index

In [17]:
electricity_data = pd.read_csv('data/electricity.csv', index_col=0)
formula = "Electricity ~ C(Lighting) + C(TV) + C(Cooking) + C(Refrigeration) + C(AC) + C(Urban) + Income"

In [18]:
electricity_data.head()

,Lighting,TV,Cooking,Refrigeration,AC,Urban,Income,Electricity
0,1,1,1,1,0,0,16000.0,110.0
1,1,1,0,1,0,0,4000.0,80.0
2,1,1,0,1,0,0,6000.0,47.0
3,1,1,0,0,0,0,6300.0,17.0
4,1,1,0,0,0,0,5000.0,17.0


In [19]:
model_elec = sm.WLS.from_formula(formula, electricity_data)
model_results_elec = model_elec.fit()

In [20]:
model_results_elec.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:            Electricity   R-squared:                       0.516
Model:                            WLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     2519.
Date:                Tue, 24 Oct 2017   Prob (F-statistic):               0.00
Time:                        12:05:46   Log-Likelihood:                -96932.
No. Observations:               16522   AIC:                         1.939e+05
Df Residuals:                   16514   BIC:                         1.939e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 3.3000     18.699      0.176      0.860     -33.351      39.951
C(Lighting)[T.1]          0.8257     18.668      0.044      0.965     -35.765      37.416
C(TV)[T.1]               18.7899      1.760     10.678      0.000      15.341      22.239
C(Cooking)[T.1]          28.8862      1.969     14.671      0.000      25.027      32.746
C(Refrigeration)[T.1]    59.2432      1.556     38.073      0.000      56.193      62.293
C(AC)[T.1]              203.3226      3.130     64.956      0.000     197.187     209.458
C(Urban)[T.1]            24.5935      1.391     17.680      0.000      21.867      27.320
Income                    0.0014    4.1e-05     34.765      0.000       0.001       0.002
==============================================================================
Omnibus:                    20742.858   Durbin-Watson:                   1.789
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9719176.595
Skew:                           6.463   Prob(JB):                         0.00
Kurtosis:                     121.115   Cond. No.                     8.75e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.75e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
params_elec = change_index(model_results_elec.params)
bse_elec = change_index(model_results_elec.bse)
elec = pd.concat([params_elec, bse_elec], axis=1)
elec.columns = ['co_mu', 'co_sd']

In [22]:
elec.loc['Lighting', 'p'] = (electricity_data.Lighting == 1).sum() / electricity_data.shape[0]
elec.loc['TV', 'p'] = (electricity_data.TV == 1).sum() / electricity_data.shape[0]
elec.loc['Cooking', 'p'] = (electricity_data.Cooking == 1).sum() / electricity_data.shape[0]
elec.loc['Refrigeration', 'p'] = (electricity_data.Refrigeration == 1).sum() / electricity_data.shape[0]
elec.loc['AC', 'p'] = (electricity_data.AC == 1).sum() / electricity_data.shape[0]
elec.loc['Urban', 'p'] = (electricity_data.Urban == 1).sum() / electricity_data.shape[0]

In [23]:
elec.loc[:, 'mu'] = np.nan
elec.loc[:, 'sd'] = np.nan
elec.loc['Intercept', 'p'] = elec.loc['Intercept', 'co_mu']
elec.loc['Intercept', ['co_mu', 'co_sd']] = np.nan

In [24]:
elec.loc[:, 'dis'] = 'Bernoulli'
elec.loc['Income', 'dis'] = 'None'
elec.loc['Intercept', 'dis'] = 'Deterministic'

In [25]:
elec.loc[:, 'ub'] = np.nan
elec.loc[:, 'lb'] = np.nan
elec.loc['Income', 'ub'] = np.inf
elec.loc['Income', 'lb'] = 0

In [26]:
elec.index = ['e_'+i for i in elec.index]

In [27]:
elec.to_csv('data/table_elec.csv')

In [28]:
elec

,co_mu,co_sd,p,mu,sd,dis,ub,lb
e_Intercept,NaN,NaN,3.299984,NaN,NaN,Deterministic,NaN,NaN
e_Lighting,0.825662,18.667601,0.998729,NaN,NaN,Bernoulli,NaN,NaN
e_TV,18.789909,1.759621,0.782774,NaN,NaN,Bernoulli,NaN,NaN
e_Cooking,28.886242,1.968938,0.167474,NaN,NaN,Bernoulli,NaN,NaN
e_Refrigeration,59.243236,1.556048,0.436812,NaN,NaN,Bernoulli,NaN,NaN
e_AC,203.322615,3.130158,0.059375,NaN,NaN,Bernoulli,NaN,NaN
e_Urban,24.593500,1.391044,0.550236,NaN,NaN,Bernoulli,NaN,NaN
e_Income,0.001426,0.000041,NaN,NaN,NaN,None,inf,0.0


<div class="image123">
    <div class="imgContainer">
        <img src="./logos/UNEnvironment.png" alt="UNEP logo" style="width:200px">
    </div>
    <div class="imgContainer">
        <img src="./logos/GI-REC.png" alt="GI_REC logo" style="width:200px">
    </div>
</div>

# 1.c Micro-level Electricity demand model

[UN Environment](http://www.unep.org/)

[Home](Welcome.ipynb)

[Next](Ad_MCMC_water.ipynb) (1.d) Micro-level Water demand model